This file imports an EXCEL model (here template example from BW25 sharepoint folder) and matches it with the background database in the project. For this repository we are currently working with ecoinvent310clca

After successfully importing a database to a Brightway project, the database is stored in the project and doesn't have to be imported every time we open the repository. Unless we make changes to the background database of course

# EXCEL Model importer

#### 1. Import packages

In [1]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
from bw2data import databases
import bw2io as bi
from bw2io import ExcelImporter
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods
import argparse
import bw2data as bd
import os

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

#### 2. Set project and see databases

In [2]:
# define a project where we install the databases and work in this script
bd.projects.set_current('LCA_Toolbox')

In [3]:
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	bw25_db
	ecoinvent-3.10-biosphere
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential

In [4]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
el_bio3 = bd.Database('biosphere3')

#### 3. Import the EXCEL file containing the model 

In [5]:
# Get user's home directory (C:\Users\USERNAME)
home = os.path.expanduser("~")

In [6]:
# Here you need to change the path to your local path where you have stored the excel file, this can also be on your sharepoint. 

excel_db = bi.ExcelImporter(os.path.join(
    home,
    "OneDrive - 2.-0 LCA Consultants ApS",  # This stays the same for all users
    "Intranet - Brightway2",
    "excel_model_example",
    "bw25_model_example.xlsx"
))

Extracted 1 worksheets in 0.04 seconds


#### 4. Importing means matching with background databases and writing the database

In [7]:
excel_db.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 39.20 seconds


In [8]:
excel_db.match_database(fields=['name','location'])

Applying strategy: link_iterable_by_fields


In [9]:
excel_db.match_database(db_name='ecoinvent310clca',
                        fields=['name','location','unit'])

StrategyError: Can't find external database ecoinvent310clca

In [ ]:
excel_db.match_database(db_name='ecoinvent-3.10-biosphere',
                        fields=['name','categories','unit'])

In [ ]:
excel_db.match_database(db_name='biosphere3',
                        fields=['name','categories','unit'])

In [ ]:
# check foreground imports for unlinked processes
pd.DataFrame(excel_db.unlinked)

In [ ]:
# if there are unlinked exchanges, this helps localizing them in order to fix them
list(excel_db.unlinked)


In [ ]:
excel_db.write_database()

#### 5. Store data in a dataframe so it can be analysed in Brightway 2.5

In [ ]:
db_template = bd.Database('bw25_db')


In [ ]:
bd.databases

In [ ]:
data_db_template = [activity.as_dict() for activity in db_template]


In [ ]:
df_db_template = pd.DataFrame(data_db_template)


In [ ]:
from IPython.display import display
display(df_db_template)

In [ ]:
# Get all activities and products
activities = set(db_template)  # All activities in the database

# Get all products linked via production exchanges
products = set([
    exc.input for act in db_template
    for exc in act.exchanges() if exc.input == act.key
])

# Identify mismatches
extra_activities = activities - products
extra_products = products - activities

print("Extra activities (no matching products):", extra_activities)
print("Extra products (no matching activities):", extra_products)

In [ ]:
# what also works to show all activities in a database is the following
for act in db_template:
    print(act)